# Functional Connectivity Sliding Window Preprocessing

We are going to make "chunks" of anxiety per subject that we can then align to fMRI data and calculate the functional connectivity. In order to standardize our feature sizes we will make window lengths of fMRI_TR, give a sliding stride, and then calculate mean anixety for the label for each chunk. We will prepare them in two different ways, one that can be used for regression (predicting continuous anxiety values) and one for classification (high vs low anxiety). Note these anxiety values are already z scored. The values from the reviewers are from 0 to 100 and then consensus annotations are calculated from the z scores. We are going to add a minmaxscaled value to each of the CSVs because that is better for some neural networks. Likely regression is best to use the continuous values and then minmax is good for binning for classification. 

In [42]:
# code to add the minmaxscaler values 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

anxiety_dir = "anxiety_aligned_csv"

for filename in os.listdir(anxiety_dir):
    if filename.endswith(".csv"):
        filepath = os.path.join(anxiety_dir, filename)

        df = pd.read_csv(filepath)

        anxiety_col = df.columns[2]

        scaler = MinMaxScaler()
        df["scaled_anxiety"] = scaler.fit_transform(df[[anxiety_col]])

        df.to_csv(filepath, index=False)
        print(f"Updated: {filename}")

Updated: sub-S25_Chatter_anxiety_aligned.csv
Updated: sub-S28_LessonLearned_anxiety_aligned.csv
Updated: sub-S20_Chatter_anxiety_aligned.csv
Updated: sub-S09_Chatter_anxiety_aligned.csv
Updated: sub-S25_LessonLearned_anxiety_aligned.csv
Updated: sub-S11_LessonLearned_anxiety_aligned.csv
Updated: sub-S02_LessonLearned_anxiety_aligned.csv
Updated: sub-S10_Chatter_anxiety_aligned.csv
Updated: sub-S13_LessonLearned_anxiety_aligned.csv
Updated: sub-S27_LessonLearned_anxiety_aligned.csv
Updated: sub-S06_Chatter_anxiety_aligned.csv
Updated: sub-S15_Chatter_anxiety_aligned.csv
Updated: sub-S03_Chatter_anxiety_aligned.csv
Updated: sub-S21_Chatter_anxiety_aligned.csv
Updated: sub-S08_Chatter_anxiety_aligned.csv
Updated: sub-S29_LessonLearned_anxiety_aligned.csv
Updated: sub-S24_LessonLearned_anxiety_aligned.csv
Updated: sub-S24_Chatter_anxiety_aligned.csv
Updated: sub-S01_LessonLearned_anxiety_aligned.csv
Updated: sub-S32_Chatter_anxiety_aligned.csv
Updated: sub-S10_LessonLearned_anxiety_aligned

# Create the sliding windows 

We are going to create CSVs for each subject that says when the window starts and what the mean z scored anxiety is as well as the binned anxiety level. We will do it based on mean and median. 

In [2]:
from utils.sliding_windows import create_windows 
import os

window_size = 30
step_size = 5 

anxiety_dir = "anxiety_aligned_csv"
output_dir = "LessonLearned_sliding_windows_csv"


for subj_num in range(1, 33):
    subj_id = f"sub-S{subj_num:02d}"  # ensures sub-S01, sub-S02, ..., sub-S32
    input_file = f"{subj_id}_LessonLearned_anxiety_aligned.csv"
    input_path = os.path.join(anxiety_dir, input_file)

    if not os.path.exists(input_path):
        print(f"Skipping {subj_id} — no file found.")
        continue

    output_path = os.path.join(output_dir, f"{subj_id}_anxiety_windows.csv")
    create_windows(
        anxiety_csv_path=input_path,
        output_csv_path=output_path,
        window_size=window_size,
        step_size=step_size
    )


KeyError: 0